In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import sys
import seaborn as sns
import matplotlib.pyplot as plt

np.set_printoptions(threshold=sys.maxsize)
np.random.seed(0)

mu = 0
sigma = 1
exp_effect = np.round(np.arange(-1, 1.01, 0.01),2)
# exp_effect = [0.1]
sample_size = np.round(np.arange(1,101,1))
# sample_size = [100]
n = 1000 # n iterations over each effect size 
upper_one = stats.norm.ppf(q = 0.95)
upper_two = stats.norm.ppf(q = 0.975)

#create matrices to contain T:test statistics, Decision_one, Decision_two: test decisions for one- and two-sided tests, respectively
T = np.array([[[0.0]*n]*len(exp_effect)] * len(sample_size)) 
Decision_one = np.array([[[0]*n]*len(exp_effect)] * len(sample_size)) 
Decision_two = np.array([[[0]*n]*len(exp_effect)] * len(sample_size))

In [ ]:
# columns: increase effect size (mu-mean)
# hues: increase sample size
# rows: repeat 10 times, on random data 

for j in range(0, len(exp_effect)):
    for k in range(0, len(sample_size)):
        for i in range(0,n):       
            sample = np.random.normal(loc=exp_effect[j], size=sample_size[k], scale=sigma)
            T[k,j,i] = (((np.mean(sample) - mu)/sigma)*np.sqrt(sample_size[k])).astype(float)
            # positive = 1, negative = 0
            if(T[k,j,i] < upper_one):
                Decision_one[k,j,i] = 0
            else:
                Decision_one[k,j,i] = 1
            if (T[k,j,i] < upper_two):
                Decision_two[k,j,i] = 0
            else:
                Decision_two[k,j,i] = 1   

In [ ]:
def mean(data):
    n = len(data)
    mean = sum(data) / n
    return mean

In [ ]:
def variance(data):
    # Number of observations
    n = len(data)
    # Mean of the data
    mean = sum(data) / n
    # Square deviations
    deviations = [(x - mean) ** 2 for x in data]
    # Variance
    variance = sum(deviations) / n
    return variance

In [ ]:
probability_one = np.array([[0.0]*len(exp_effect)] * len(sample_size))
probability_two = np.array([[0.0]*len(exp_effect)] * len(sample_size))
deviation_one = np.array([[0.0]*len(exp_effect)] * len(sample_size))
deviation_two = np.array([[0.0]*len(exp_effect)] * len(sample_size))

for k in range(0,len(sample_size)):
    for j in range(0,len(exp_effect)):
        probability_one[k,j] = mean(Decision_one[k,j,:])
        probability_two[k,j] = mean(Decision_two[k,j,:])
        deviation_one[k,j] = variance(Decision_one[k,j,:])
        deviation_two[k,j] = variance(Decision_two[k,j,:])

In [ ]:
df_one = pd.DataFrame(probability_one, index=sample_size, columns=exp_effect)
df_two = pd.DataFrame(probability_two, index=sample_size, columns=exp_effect)
df_diff = df_one-df_two
df_std1 = pd.DataFrame(deviation_one, index=sample_size, columns=exp_effect)
df_std2 = pd.DataFrame(deviation_two, index=sample_size, columns=exp_effect)
df_stddiff = df_std1-df_std2

In [ ]:
df_one.iloc[:,100:201]

In [ ]:
sns.heatmap(df_one.iloc[:,100:201], cmap='flare')
plt.savefig(r'C:\Users\phili\OneDrive\Dokumente_One Drive\KIT\Bachelorarbeit\plots\chapter5\plot2.png')

In [ ]:
sns.set_theme(palette='pastel')
g = sns.lineplot(x=exp_effect, y=df_one.iloc[0,:], label='sample size = 1')
sns.lineplot(x=exp_effect, y=df_two.iloc[24,:], label='sample size = 25')
sns.lineplot(x=exp_effect, y=df_one.iloc[39,:], label='sample size = 40')
sns.lineplot(x=exp_effect, y=df_one.iloc[99,:], label='sample size = 100')
g.set(xlabel='effect size', ylabel='p(one-tailed = positive=')

In [ ]:
sns.heatmap(df_two.iloc[:,100:201], cmap='flare')
plt.savefig(r'C:\Users\phili\OneDrive\Dokumente_One Drive\KIT\Bachelorarbeit\plots\chapter5\plot3.png')

In [ ]:
sns.set_theme(palette='colorblind')
g = sns.lineplot(x=exp_effect, y=df_two.iloc[0,:], label='sample size = 1')
sns.lineplot(x=exp_effect, y=df_two.iloc[24,:], label='sample size = 25')
sns.lineplot(x=exp_effect, y=df_two.iloc[39,:], label='sample size = 40')
sns.lineplot(x=exp_effect, y=df_two.iloc[99,:], label='sample size = 100')
g.set(xlabel='effect size', ylabel='p(two-tailed = positive)')

In [ ]:
sns.heatmap(df_diff, cmap='flare')

In [ ]:
with sns.axes_style('darkgrid'):
    g = sns.scatterplot(x=exp_effect, y=df_diff.iloc[4,:], color='dodgerblue', label='sample size = 5')
    sns.scatterplot(x=exp_effect, y=df_diff.iloc[24,:], color='orangered',label='sample size = 25')
    sns.scatterplot(x=exp_effect, y=df_diff.iloc[49,:], color='navy', label='sample size = 50')
    sns.scatterplot(x=exp_effect, y=df_diff.iloc[99,:], color='firebrick', label='sample size = 100')
    g.set(xlabel='absolute effect size', ylabel='probability difference')
plt.savefig(r'C:\Users\phili\OneDrive\Dokumente_One Drive\KIT\Bachelorarbeit\plots\chapter5\plot4.png')
print(stats.normaltest(df_one.iloc[:,4]))
print(stats.normaltest(df_diff.iloc[24,:]))
print(stats.normaltest(df_diff.iloc[49,:]))
print(stats.normaltest(df_diff.iloc[99,:]))

In [ ]:
with sns.axes_style('darkgrid'):
    ax = sns.scatterplot(x=exp_effect, y=df_std1.iloc[0,:], color='seagreen', label='one-tailed, sample = 1')
    sns.scatterplot(x=exp_effect, y=df_std2.iloc[0,:], color='lightseagreen', label='two-tailed, sample = 1')
    sns.scatterplot(x=exp_effect, y=df_std1.iloc[9,:], color='navy', label='one-tailed, sample = 10')
    sns.scatterplot(x=exp_effect, y=df_std2.iloc[9,:], color='dodgerblue', label='two-tailed, sample = 10')
    sns.scatterplot(x=exp_effect, y=df_std1.iloc[99,:], color='firebrick', label='one-tailed, sample = 100')
    sns.scatterplot(x=exp_effect, y=df_std2.iloc[99,:], color='orangered', label='two-tailed, sample = 100')
    ax.set(xlabel='ansolute effect size', ylabel='variance')
plt.savefig(r'C:\Users\phili\OneDrive\Dokumente_One Drive\KIT\Bachelorarbeit\plots\chapter5\plot5.png')

In [ ]:
sns.heatmap(df_std1, cmap='flare')

In [ ]:
sns.heatmap(df_std2, cmap='flare')

In [ ]:
sample_size = np.arange(start=500, stop=1000, step=1)
effect_size = np.round(np.arange(start=-1, stop=1, step=0.01),2)
iteration = 100
t = np.array([[[0.0]*n]*len(exp_effect)] * len(sample_size)) 
d_one = np.array([[[0]*n]*len(exp_effect)] * len(sample_size)) 
d_two = np.array([[[0]*n]*len(exp_effect)] * len(sample_size))

for j in range(0, len(effect_size)):
    for k in range(0, len(sample_size)):
        for i in range(0,iteration):       
            sample = np.random.normal(loc=effect_size[j], size=sample_size[k], scale=sigma)
            t[k,j,i] = (((np.mean(sample) - mu)/sigma)*np.sqrt(sample_size[k])).astype(float)
            # positive = 1, negative = 0
            if(t[k,j,i] < upper_one):
                d_one[k,j,i] = 0
            else:
                d_one[k,j,i] = 1
            if (t[k,j,i] < upper_two):
                d_two[k,j,i] = 0
            else:
                d_two[k,j,i] = 1   

In [ ]:
s_one = np.array([[0.0]*len(effect_size)] * len(sample_size))
s_two = np.array([[0.0]*len(effect_size)] * len(sample_size))
prob_one = np.array([[0.0]*len(effect_size)] * len(sample_size))
prob_two = np.array([[0.0]*len(effect_size)] * len(sample_size))

for j in range(0, len(effect_size)):
    for k in range(0,len(sample_size)):
        for i in range(0,iteration):
            s_one[k,j] = s_one[k,j] + d_one[k,j,i] 
            s_two[k,j] = s_two[k,j] + d_two[k,j,i]
            prob_one[k,j] = s_one[k,j] / iteration
            prob_two[k,j] = s_two[k,j] / iteration
dfOne = pd.DataFrame(prob_one, index=sample_size, columns=effect_size)
dfTwo = pd.DataFrame(prob_two,index=sample_size, columns=effect_size)
dfDiff = dfOne - dfTwo

In [ ]:
sns.heatmap(dfDiff, cmap='flare')